In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Mount Google Drive
drive.mount('/content/drive')

# Path to the directory containing your images in Google Drive
drive_directory_path = '/content/drive/My Drive/JPG'

# Data Preparation for Images
data_dir = '/content/drive/My Drive/JPG'  # Path to images folder in Drive
images = []
labels = []

##############
############## THIS SECTION IS OPTIONAL IF YOU HAVENT PREPROCESSED THE DATASET YET
for filename in os.listdir(data_dir):
    if filename.endswith('.png'):
        img = cv2.imread(os.path.join(data_dir, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (100, 100))  # Resize image to desired dimensions
        img = img / 255.0  # Normalize pixel values
        images.append(img)


###############

        # Label processing - 1 for presence of human head, 0 for absence
        label = 1 if 'human_head' in filename else 0
        labels.append(label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split data into train, validation, test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Build a more complex CNN Model with modified dropout rates
output_classes = 2  # Number of output classes (binary in this case)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 1), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.9),  # Higher dropout rate
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.6),  # Moderate dropout rate
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.8),  # Higher dropout rate
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),  # Standard dropout rate
    tf.keras.layers.Dense(output_classes, activation='softmax')  # Softmax activation for multi-class
])

# Compile the model for multi-class classification
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train[..., np.newaxis], y_train, epochs=15, batch_size=64, validation_data=(X_val[..., np.newaxis], y_val))

# Evaluate the model
loss, accuracy = model.evaluate(X_test[..., np.newaxis], y_test)
print(f'Test Accuracy: {accuracy}')

# Quantize the model
quantized_model = tf.keras.models.clone_model(model)
quantized_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(X_train[..., np.newaxis]).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
quantized_tflite_model = converter.convert()

# Save the quantized model to a file
with open('/content/drive/My Drive/quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)
